#### Spark

In [1]:
from pyspark.sql import SparkSession

# Cria uma SparkSession
spark = SparkSession.builder.appName("AnaliseRH").getOrCreate()

#### Carregando os dados originais

In [12]:
# Definir diretórios
dados_originais = r"dados/"
dados_processados = r"dados_processados/"

# Carregar os datasets
df_saude_mental = spark.read.csv(f"{dados_originais}Impacto_do_Trabalho_Remoto_na_Saude_Mental.csv", header=True, inferSchema=True)
df_bem_estar_feedback = spark.read.csv(f"{dados_originais}Programas_de BemEstar_e_Feedback_dos_Funcionários.csv", header=True, inferSchema=True)
df_prog_carreira = spark.read.csv(f"{dados_originais}Progresssao_de_Carreira_do_Funcionário.csv", header=True, inferSchema=True)

In [13]:
# Renomear colunas
df_saude_mental = df_saude_mental.withColumnRenamed("Employee_ID", "ID_Empregado") \
    .withColumnRenamed("Age", "Idade") \
    .withColumnRenamed("Gender", "Genero") \
    .withColumnRenamed("Job_Role", "Cargo") \
    .withColumnRenamed("Industry", "Industria") \
    .withColumnRenamed("Years_of_Experience", "Anos_Experiencia") \
    .withColumnRenamed("Work_Location", "Local_Trabalho") \
    .withColumnRenamed("Hours_Worked_Per_Week", "Horas_Trabalhadas_Semana") \
    .withColumnRenamed("Number_of_Virtual_Meetings", "Numero_Reunioes_Virtuais") \
    .withColumnRenamed("Work_Life_Balance_Rating", "Avaliacao_Equilibrio_Vida") \
    .withColumnRenamed("Stress_Level", "Nivel_Estresse") \
    .withColumnRenamed("Mental_Health_Condition", "Condicao_Saude_Mental") \
    .withColumnRenamed("Access_to_Mental_Health_Resources", "Acesso_Recursos_Saude_Mental") \
    .withColumnRenamed("Productivity_Change", "Mudanca_Produtividade") \
    .withColumnRenamed("Social_Isolation_Rating", "Avaliacao_Isolamento_Social") \
    .withColumnRenamed("Satisfaction_with_Remote_Work", "Satisfacao_Trabalho_Remoto") \
    .withColumnRenamed("Company_Support_for_Remote_Work", "Suporte_Empresa_Trabalho_Remoto") \
    .withColumnRenamed("Physical_Activity", "Atividade_Fisica") \
    .withColumnRenamed("Sleep_Quality", "Qualidade_Sono") \
    .withColumnRenamed("Region", "Regiao")

df_bem_estar_feedback = df_bem_estar_feedback.withColumnRenamed("Employee_ID", "ID_Empregado") \
    .withColumnRenamed("Program_ID", "ID_Programa") \
    .withColumnRenamed("Program_Name", "Nome_Programa") \
    .withColumnRenamed("Participation_Status", "Status_Participacao") \
    .withColumnRenamed("Feedback_Score", "Pontuacao_Feedback") \
    .withColumnRenamed("Improvement_Noticed", "Melhoria_Observada")

df_prog_carreira = df_prog_carreira.withColumnRenamed("Employee_ID", "ID_Empregado") \
    .withColumnRenamed("Promotion_Date", "Data_Promocao") \
    .withColumnRenamed("Old_Role", "Cargo_Anterior") \
    .withColumnRenamed("New_Role", "Novo_Cargo") \
    .withColumnRenamed("Salary_Change", "Mudanca_Salarial") \
    .withColumnRenamed("Role_Satisfaction", "Satisfacao_Cargo")

#### Exibindo informações básicas

In [14]:
# Exibir informações básicas
def exibir_info(df, nome):
    print(f"\n{nome} Info:")
    df.printSchema()
    df.show(5)

dfs = {"Saude Mental": df_saude_mental, "Bem-estar": df_bem_estar_feedback, "Progresso Carreira": df_prog_carreira}
for nome, df in dfs.items():
    exibir_info(df, nome)


Saude Mental Info:
root
 |-- ID_Empregado: string (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Cargo: string (nullable = true)
 |-- Industria: string (nullable = true)
 |-- Anos_Experiencia: integer (nullable = true)
 |-- Local_Trabalho: string (nullable = true)
 |-- Horas_Trabalhadas_Semana: integer (nullable = true)
 |-- Numero_Reunioes_Virtuais: integer (nullable = true)
 |-- Avaliacao_Equilibrio_Vida: integer (nullable = true)
 |-- Nivel_Estresse: string (nullable = true)
 |-- Condicao_Saude_Mental: string (nullable = true)
 |-- Acesso_Recursos_Saude_Mental: string (nullable = true)
 |-- Mudanca_Produtividade: string (nullable = true)
 |-- Avaliacao_Isolamento_Social: integer (nullable = true)
 |-- Satisfacao_Trabalho_Remoto: string (nullable = true)
 |-- Suporte_Empresa_Trabalho_Remoto: integer (nullable = true)
 |-- Atividade_Fisica: string (nullable = true)
 |-- Qualidade_Sono: string (nullable = true)
 |-- Regiao: string (n

In [15]:
# Exibir contagem de linhas e colunas (linhas, colunas)
print("Saúde Mental", (df_saude_mental.count(), len(df_saude_mental.columns)))
print("Bem-estar", (df_bem_estar_feedback.count(), len(df_bem_estar_feedback.columns)))
print("Progresso de Carreira", (df_prog_carreira.count(), len(df_prog_carreira.columns)))

Saúde Mental (5000, 20)
Bem-estar (1000, 6)
Progresso de Carreira (500, 6)


#### Tratando valores ausentes, removendo duplicatas e excluindo colunas irrelevantes.

In [19]:
from pyspark.sql.functions import col, count, when, mean, unix_timestamp

# Tratamento de valores ausentes
def tratar_nulos(df):
    for col_name in df.columns:
        # Pular as colunas ID_Empregado e Data_Promocao
        if col_name in ["ID_Empregado", "Data_Promocao"]:
            continue
        
        # Verificar se a coluna não é do tipo string
        if df.schema[col_name].dataType.simpleString() != 'string':
            # Se for do tipo DATE, converta para número (dias desde 1970-01-01)
            if df.schema[col_name].dataType.simpleString() == 'date':
                df = df.withColumn(
                    col_name, 
                    unix_timestamp(col(col_name)).cast("double")
                )
            # Calcular a média para colunas numéricas
            media_valor = df.select(mean(col(col_name))).collect()[0][0]
            df = df.fillna({col_name: media_valor})
        else:
            # Preencher valores nulos com 'Desconhecido' para colunas de texto
            df = df.fillna({col_name: 'Desconhecido'})
    
    return df

df_saude_mental = tratar_nulos(df_saude_mental)
df_bem_estar_feedback = tratar_nulos(df_bem_estar_feedback)
df_prog_carreira = tratar_nulos(df_prog_carreira)

In [20]:
dfs = {"Saude Mental": df_saude_mental, "Bem-estar": df_bem_estar_feedback, "Progresso Carreira": df_prog_carreira}
for nome, df in dfs.items():
    exibir_info(df, nome)


Saude Mental Info:
root
 |-- ID_Empregado: string (nullable = false)
 |-- Idade: integer (nullable = true)
 |-- Genero: string (nullable = false)
 |-- Cargo: string (nullable = false)
 |-- Industria: string (nullable = false)
 |-- Anos_Experiencia: integer (nullable = true)
 |-- Local_Trabalho: string (nullable = false)
 |-- Horas_Trabalhadas_Semana: integer (nullable = true)
 |-- Numero_Reunioes_Virtuais: integer (nullable = true)
 |-- Avaliacao_Equilibrio_Vida: integer (nullable = true)
 |-- Nivel_Estresse: string (nullable = false)
 |-- Condicao_Saude_Mental: string (nullable = false)
 |-- Acesso_Recursos_Saude_Mental: string (nullable = false)
 |-- Mudanca_Produtividade: string (nullable = false)
 |-- Avaliacao_Isolamento_Social: integer (nullable = true)
 |-- Satisfacao_Trabalho_Remoto: string (nullable = false)
 |-- Suporte_Empresa_Trabalho_Remoto: integer (nullable = true)
 |-- Atividade_Fisica: string (nullable = false)
 |-- Qualidade_Sono: string (nullable = false)
 |-- Regia